## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 우리의 앱에서 중요한 요소 중 하나는 와인을 추천하고, 이를 실제로 구매할 수 있는 링크에 연결하는 것입니다. 또, 실제 와인바를 추천해주기도 합니다. 이를 위해서 실제 와인을 파는 사이트와 와인바를 데이터베이스로 만들어두어야하기 때문에 이를 위해서는 크롤링이 필요합니다.
2. 현재 와인나라와 와인앤모어에 대한 크롤링을 완료한 상태입니다. 하지만 이를 통합된 하나의 데이터베이스로 만드는 과정이 필요합니다.
- 와인나라: [winenara.json](./winenara_scrapy/winenara.json)
- 와인앤모어: [wine_and_more.json](./wine_and_more.json)
3. 와인나라에 대한 데이터를 크롤링했으나 바디, 산도 등 없는 데이터가 많았습니다. 이 때문에 X wine dataset을 이용해 부족한 부분을 추가해주어야합니다.

#### 당신의 목표는 아래와 같습니다.
1. 아래 사이트에 대한 크롤링을 진행합니다. 크롤링이 필요한 요소는 와인이나 와인바 추천에 필요하다고 생각하는 모든 것을 하면 됩니다. 또한 추가로 더 좋은 사이트의 크롤링도 좋습니다.

    <strong>와인</strong>
    - [와인나라](https://www.winenara.com/shop/main)
    - [와인앤모어](https://www.wineandmore.co.kr/)
    - [CU 편의점](https://pocketcu.co.kr/)
    - [SSG 와인장터](https://emart.ssg.com/specialStore/wineliquor/main.ssg)
    - [롯데온](https://www.lotteon.com/search/search/search.ecn?render=search&platform=pc&q=%EC%99%80%EC%9D%B8&mallId=1)


<br>

2. 이후 와인에 대한 데이터베이스를 통합합니다. 현재 아래의 schema를 생각하고 있으나 다른 것을 추가하는 것도 좋습니다.
- url(required): 해당 페이지의 url
- site_name(required): 해당 페이지의 출처(와인나라, 와인앤모어 등)
- price(required): 가격
- name: 한글 이름
- en_name: 영어 이름
- img_url(required): 이미지 url 리스트
- body: 바디감(1~5점)
- acidity: 산도(1~5점)
- tannin: 타닌(1~5점)
- sweetness: 당도(1~5점)
- alcohol: %
- wine_type(required): 와인타입(레드, 화이트 등)
- country: 원산지(미국, 독일 등)
- grape: 포도품종
- rating: vivino 점수(1~5점)
- pickup_location: 픽업장소
- vivino_link: vivino 사이트의 와인 상세페이지 url

#### 참고 사항
현재 작성된 크롤링은 아래의 프레임워크로 작업했습니다.
- [Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)
- [Selenium](https://selenium-python.readthedocs.io/)

## 와인나라 크롤링

와인나라에 있는 와인에 대한 정보를 가져오는 것을 목적으로 한다.

크롤링에는 [Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)를 사용한다.

scrapy를 사용하기 위해서 [새로운 프로젝트](./winenara_scrapy)에서 작업을 한다. 여기서 [winenara_spiders](./winenara_scrapy/tutorial/spiders/winenara_spider.py) 파일을 수정한다.

### 상품목록페이지에서 각 상품 상세페이지 링크 추출
<img src="./assets/winenara_mainpage.png" width="500" height="500">

위에서 각 상세페이지의 주소를 가져오고, 모든 상세페이지에서 정보를 가져왔다면 다음 페이지 링크를 이용해 다음 페이지로 넘어간다.

### 상세페이지에서 와인 정보 추출
<img src="./assets/winenara.png" width="1000" height="500">

위의 이미지는 와인나라의 상세페이지이다. 상세페이지로부터 아래의 값들을 가져온다.
- url: 해당 페이지의 url
- price: 가격
- name: 한글 이름
- en_name: 영어 이름
- img_url: 이미지 url 리스트
- features: 와인에 대한 정보 딕셔너리(바디, 산도, 타닌, 알코올 등)
- tag: 와인에 대한 정보 리스트(와인의 종류, 원산지, 포도품종)
- rating: vivino 점수(1~5점)
- vivino_link: vivino url (vivino는 전세계의 와인 데이터베이스 웹페이지이다.)

예를 들어 위의 이미지에 대해서는 아래와 같이 만들어진다.
```json
{"url": "https://www.winenara.com/shop/product/product_view?product_cd=29E033",
 "price": "50,000원", 
 "name": "SET)페데럴리스트 카베르네 소비뇽 원통 패키지", 
 "en_name": "SET)THE FEDERALIST CABERNET SAUVIGNON", 
 "img_url": ["https://www.winenara.com/uploads/product/550/e0705cecad48a9e01710b6fbf7b2f8c5.png"], 
 "features": {"바디": "중간"},
 "feature_img_url": ["https://www.winenara.com/uploads/editor/202212081007511218373150.jpg"],
 "tag": ["레드", "미국", "카베르네 소비뇽"], 
 "rating": "3.8", 
 "vivino_link": "https://www.vivino.com/federalist-cabernet-sauvignon/w/3048981", "images": [{"url": "https://www.winenara.com/uploads/product/550/e0705cecad48a9e01710b6fbf7b2f8c5.png", "path": "product\\550\\e0705cecad48a9e01710b6fbf7b2f8c5.png", "checksum": "e04ed429fd54db2478136c3e03ab592f", "status": "downloaded"}]}
```

----------------- 코드 실행 ------------------

In [1]:
%cd winenara_scrapy

c:\Users\chois\Desktop\chatwine\winenara_scrapy


In [3]:
%cd tutorial

c:\Users\chois\Desktop\chatwine\winenara_scrapy\tutorial


크롤링 과정에서 img_url의 이미지가 로컬에 저장된다. 이를 위해 [settings.py](./winenara_scrapy/tutorial/settings.py)에 이미지가 저장될 경로를 설정한다. settings.py에 경로를 변경해서 아래 코드를 추가해준다.
```python
ITEM_PIPELINES = {'tutorial.pipelines.CustomImagesPipeline': 1}
IMAGES_STORE = 'C:/Users/chois/Desktop/Audrey/chatwine/winenara_scrapy/assets/img'
```

또한 이미지 저장에 있어서 이미지의 이름을 변경해주기 위해 [pipelines.py](./winenara_scrapy/tutorial/pipelines.py)에 CustomImagePipeline 클래스를 추가해준다. 코드는 아래와 같다.
```python
import os
from urllib.parse import urlparse

import scrapy
from scrapy.pipelines.images import ImagesPipeline

class CustomImagesPipeline(ImagesPipeline):
    def file_path(self, request, response=None, info=None, *, item=None):
        path = urlparse(request.url).path
        path_parts = path.split('/')
        filename = os.path.join(*path_parts[-3:])
        return filename

    def get_media_requests(self, item, info):
        if 'img_url' in item:
            for image_url in item['img_url']:
                yield scrapy.Request(image_url)
```

아래 코드를 실행하여 얻은 결과는 [winenara.json](./winenara_scrapy/winenara.json)에서 확인할 수 있다.

In [ ]:
!scrapy crawl winenara -o new_winenara.json # winenara spider를 실행하여 결과를 winenara.json에 저장, 터미널에서 실행해야해요.

아래는 파일 feature_img_url을 [저장 경로](./winenara_scrapy/assets/img/feature_img/)에 저장하는 코드이다.(위의 CustomImagePipeline을 수정하면 된다. 그런데 깜빡했다!)

In [ ]:
import json
import os
import requests

# JSON 파일 열기
with open('./new_winenara.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 저장 경로 지정
save_path = '../assets/img/feature_img'

# 모든 아이템에 대해 반복
for item in data:
    image_url = item['feature_img_url'][0]  # feature_img_url에서 이미지 URL 가져오기
    image_name = item['name'] + '.png'  # 이미지 이름 지정하기

    # URL에서 이미지 다운로드하기
    try:
        response = requests.get(image_url)

        # 이미지를 파일로 저장하기
        with open(os.path.join(save_path, image_name), 'wb') as img_file:
            img_file.write(response.content)
    except:
        print('Cannot download image from:', item['name'])
        continue   


아래는 저장한 feature_img로 부터 OCR을 통해 정보를 가져오는 단계이다. 
feature 이미지는 아래와 같다.

<img src="./assets/winenara_feature_img.png" width="500" height="1000">

가져올 수 있는 feature에는 아래와 같은 값이 있다.
- 대표 맛과 향
- 핸들링
- 와이너리이야기
- 페어링

In [103]:
import os
import configparser

In [112]:
%cd ../

c:\Users\chois\Desktop\chatwine


In [113]:
%pwd # 'c:\\Users\\chois\\Desktop\\chatwine'

'c:\\Users\\chois\\Desktop\\chatwine'

In [114]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [115]:
NAVER_OCR_API_URL = config['NAVER_OCR']['API_URL']
NAVER_OCR_SECRET_KEY = config['NAVER_OCR']['SECRET_KEY']

In [117]:
import requests
import uuid
import time
import json
import os
from tqdm import tqdm
import re

class NaverOCR:
    def __init__(self, save_root_path='../assets/ocr_result'):
        self.api_url = NAVER_OCR_API_URL
        self.secret_key = NAVER_OCR_SECRET_KEY
        self.save_root_path = save_root_path
        os.makedirs(save_root_path, exist_ok=True)

        request_json = {
        'images': [
        {
        'format': 'jpg',
        'name': 'demo'
        }
        ],
        'requestId': str(uuid.uuid4()),
        'version': 'V2',
        'timestamp': int(round(time.time() * 1000))
        }

        self.payload = {'message': json.dumps(request_json).encode('UTF-8')}
        
        self.headers = {
        'X-OCR-SECRET': self.secret_key
        }

    def get_ocr(self, img_file):
        files = [
        ('file', open(img_file,'rb'))
        ]
        filename = os.path.basename(img_file)
        response = requests.request("POST", self.api_url, headers=self.headers, data = self.payload, files = files)
        with open(os.path.join(self.save_root_path, filename.replace('.png', '.json')), 'w', encoding='utf-8') as f:
            f.write(response.text)
        return response.text

In [46]:
ocr = NaverOCR()

아래 코드를 통해서 feature_img를 Naver OCR에 통과시키고, 그 response를 ./winenara_scrapy/assets/raw_ocr_results/ 에 저장한다.

In [55]:
for img_file_path in tqdm(img_file_path_list):
    ocr.get_ocr(os.path.join(img_file_root_path, img_file_path))

  0%|          | 0/472 [00:00<?, ?it/s]

100%|██████████| 472/472 [14:43<00:00,  1.87s/it]


아래코드는 Naver OCR을 통해 저장한 raw response를 실제 우리가 원하는 feature로 정제하여 json으로 저장하는 코드이다. 정제된 feature는 ./winenara_scrapy/assets/ocr_results/ 에 저장된다. feature는 위에서 말한것 처럼 다음과 같다. 

- 대표 맛과 향
- 핸들링
- 와이너리이야기
- 페어링

In [102]:
def split_text_to_dict(text, keywords):
    keyword_positions = []
    
    for keyword in keywords:
        keyword_index = text.find(keyword)
        if keyword_index != -1:
            keyword_positions.append((keyword, keyword_index))

    keyword_positions.sort(key=lambda x: x[1])
    
    split_dict = {}
    current_index = 0
    for keyword, position in keyword_positions:
        if current_index != 0:
            last_keyword, _ = keyword_positions[keyword_positions.index((keyword, position))-1]
            split_dict[last_keyword] = text[current_index:position].strip()
        current_index = position + len(keyword)

    if current_index != 0:
        last_keyword, _ = keyword_positions[-1]
        split_dict[last_keyword] = text[current_index:].strip()

    if "대표 맛과 향" in split_dict:
        # Remove English letters
        split_dict["대표 맛과 향"] = re.sub(r'[a-zA-Z]', '', split_dict["대표 맛과 향"])

    if "핸들링" in split_dict:
        # Split by space, reorder, and join again
        split_dict["핸들링"] = split_dict["핸들링"].strip()
        
        special_chars = '()YG오'
        for special_char in special_chars:
            split_dict["핸들링"] = split_dict["핸들링"].replace(special_char, '')
        handling_parts = split_dict["핸들링"].split(' ')
        handling_parts = [string for string in handling_parts if string != '']

        if len(handling_parts) == 11:
            order = [0, 5, 6, 1, 7, 2, 8, 3, 9, 4, 10]
        elif len(handling_parts) == 10:
            order = [0, 5, 1, 6, 2, 7, 3, 8, 4, 9]
        elif len(handling_parts) == 9:
            order = [0, 4, 5, 1, 6, 2, 7, 3, 8]
        elif len(handling_parts) == 8:
            if '아로마' in handling_parts:
                order = [0, 3, 4, 1, 5, 6, 2, 7]
            else:
                order = [0, 4, 1, 5, 2, 6, 3, 7]
        elif len(handling_parts) == 7:
            order = [0, 3, 4, 1, 5, 2, 6]
        elif len(handling_parts) == 6:
            order = [0, 3, 1, 4, 2, 5]
        else:
            print(len(handling_parts), handling_parts)
            print(split_dict["핸들링"])
        reordered_handling_parts = [handling_parts[i] for i in order]
        
        split_dict["핸들링"] = ' '.join(reordered_handling_parts)

    return split_dict

  0%|          | 0/472 [00:00<?, ?it/s]

100%|██████████| 472/472 [00:01<00:00, 340.14it/s]


In [ ]:
keywords = ['대표 맛과 향', '핸들링', '페어링', '와이너리이야기']
raw_ocr_file_root_path = '../assets/raw_ocr_results'
ocr_file_root_path = '../assets/ocr_results'
os.makedirs(ocr_file_root_path, exist_ok=True)
raw_ocr_file_path_list = os.listdir(raw_ocr_file_root_path)

for raw_ocr_file_path in tqdm(raw_ocr_file_path_list):
    filename = os.path.basename(raw_ocr_file_path)
    with open(os.path.join(raw_ocr_file_root_path, raw_ocr_file_path), 'r', encoding='utf-8') as f:
        dictionary = json.load(f)
        unified_text = ''
        for text in dictionary['images'][0]['fields']:
            unified_text += text['inferText'] + ' '
        split_dict = split_text_to_dict(unified_text, keywords)

    with open(os.path.join(ocr_file_root_path, filename), 'w', encoding='utf-8') as f:
        json.dump(split_dict, f, ensure_ascii=False, indent=4)

### 와인나라 와인의 vivino를 통한 특징 추출

와인나라에서는 판매하는 와인에 대한 feature(바디감, 당도 등...)를 제공하고 있다. 하지만 작성이 안되어있는 와인들도 많다. 따라서 이 정보들을 가져오기 위해서 아래 두 가지의 외부 데이터를 사용할 것이다. 
1. [vivino api](https://github.com/aptash/vivino-api)
2. [X-wine dataset](https://github.com/rogerioxavier/X-Wines)

이를 위해 먼저 vivino api github을 클론한다.

In [ ]:
%cd .. # pwd가 chatwine이 되도록 함
!git clone "https://github.com/aptash/vivino-api.git"

이후 [vivino.js](./vivino-api/vivino.js)의 244 line을 일부 수정한다. 이는 각 와인 검색의 json파일이 저장될 경로를 지정하기 위함이다.
```javascript
const outFile = fs.createWriteStream('./output/'+name+'.json');
```

vivino api는 아래와 같이 사용할 수 있다.
```shell
cd vivino-api
node vivino.js --name=ad-vivum-cabernet-sauvignon --minPrice=10 --maxPrice=25
```

이것에 대한 결과는 [./output/ad-vivum-cabernet-sauvignon.json](vivino-api/output/ad-vivum-cabernet-sauvignon.json)과 같이 json파일로 얻을 수 있다.
```json
{
  "vinos": [
    {
      "name": "AD VIVUM Cabernet Sauvignon",
      "link": "https://www.vivino.com/US-CA/en/wines/2493776",
      "thumb": "https://images.vivino.com/thumbs/O_7LrDsLQSOxl2CqVfRHVA_pb_300x300.png",
      "country": "United States",
      "region": "Napa Valley",
      "average_rating": 4.4,
      "ratings": 261,
      "price": 186.16
    }
  ],
  "status": "FULL_DATA"
}
```

이제 [winenara.json](./winenara_scrapy/winenara.json)의 와인들에 대해서 vivino api를 통해 검색하는 코드를 보자.

검색을 위해서는 winenara.json의 vivino_link를 활용한다. url의 쿼리에서 와인 이름을 추출하여 이를 이용하여 검색한다. 예시는 아래와 같다.

> vivino_link: https://www.vivino.com/monopole-la-rioja-blanco-seco-clasico/w/1925448 <br>
> -> 추출한 와인이름: monopole-la-rioja-blanco-seco-clasico

In [ ]:
import json
from tqdm import tqdm

# Read the winenara json file
with open('./winenara_scrapy/winenara.json', 'r') as f:
    data = json.load(f)

In [ ]:
%cd vivino-api

아래 코드처럼 멀티프로세싱을 활용하여 와인 데이터를 검색한다. 그 결과는 "./vivino-api/output"에 저장된다.

In [ ]:
import concurrent.futures
from tqdm import tqdm

import os
file_list = os.listdir('./output')
json_list = [file.replace('.json', '') for file in file_list]

data_list   = []
for wine_data  in data:
    if wine_data['vivino_link'].split('/')[-3] not in json_list:
        data_list.append(wine_data)

def fetch_wine_data(wine_data):
    wine_name = wine_data['vivino_link'].split('/')[-3]
    !node vivino.js "--name={wine_name}"

# Use a ThreadPoolExecutor to run fetch_wine_data in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    list(tqdm(executor.map(fetch_wine_data, data_list), total=len(data_list)))

### 와인나라 와인의 X-wine 데이터셋을 통한 특징 추출

### 와인나라 와인의 데이터 통합

아래 과정은 naive하게 크롤링한 와인나라 와인의 통합된 형태로 변경하고, json을 저장하는 과정입니다.

In [118]:
import json
import pandas as pd

In [119]:
with open('./data/winenara.json', 'r', encoding='utf-8') as f:
    wine_list = json.load(f)
df = pd.DataFrame(wine_list, columns=['price', 'name', 'en_name','rating', 'url', 'tag', 'features', 'img_url', 'vivino_link'])

In [120]:
wine_type_list = ['레드', '로제', '스파클링', '화이트', '디저트', '주정강화']
country_list = ['기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주']

In [121]:
drop_idx = []
for idx in range(len(df)):
    if (df['tag'][idx][0] not in wine_type_list) or (df['tag'][idx][1] not in country_list):
        drop_idx.append(idx)
df = df.drop(drop_idx)
df = df.reset_index(drop=True)


In [122]:
df['wine_type'] = ''
df['country'] = ''
df['sweetness'] = ''
df['body'] = ''
df['alcohol'] = ''
for idx in range(len(df)):
    df['wine_type'][idx] = df['tag'][idx][0]
    df['country'][idx] = df['tag'][idx][1]
    for key, value in df['features'][idx].items():
        if key == '당도':
            df['sweetness'][idx] = value
        elif key == '알코올':
            df['alcohol'][idx] = value
        elif key == '바디':
            df['body'][idx] = value
        else:
            raise ValueError('Wrong feature key')

In [123]:
df = df.drop(columns=['tag', 'features'])

In [124]:
df['price'] = df['price'].replace('원', '', regex=True)
df['price'] = df['price'].replace(',', '', regex=True)

In [125]:
sweetness_dict = {'':-1, '드라이':1, '미디움드라이':2, '미디엄':3, '미디움스윗':4, '스윗':5}
body_dict = {'':-1, '가벼움':1, '약간가벼움':2, '중간':3, '약간무거움':4, '무거움':5}
alcohol_dict = {'':-1, '중간(12~13%)':3,'높음(14%+)':5}


In [126]:
for idx in range(len(df)):
    df['sweetness'][idx] = sweetness_dict[df['sweetness'][idx]]
    df['body'][idx] = body_dict[df['body'][idx]]
    df['alcohol'][idx] = alcohol_dict[df['alcohol'][idx]]

In [127]:
df.columns

Index(['price', 'name', 'en_name', 'rating', 'url', 'img_url', 'vivino_link',
       'wine_type', 'country', 'sweetness', 'body', 'alcohol'],
      dtype='object')

In [32]:
df.head()

,price,name,en_name,rating,url,img_url,vivino_link,wine_type,country,sweetness,body,alcohol
0,49000,모노폴 클라시코,Monopole Classico,3.8,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/monopole-la-rioja-blanc...,화이트,스페인,-1,3,-1
1,32000,슐럼베르거 로제 스페셜 브뤼,Schlumberger Rose Special Brut,3.8,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/schlumberger-spring-edi...,스파클링,독일,-1,3,-1
2,50000,SET)페데럴리스트 샤르도네 원통 패키지,SET)THE FEDERALIST CHARDONNAY,3.7,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/federalist-chardonnay-m...,화이트,미국,-1,3,-1
3,55000,베니카 트레 비니스,VENICA TRE VIGNIS,3.9,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/US-CA/en/venica-venica-...,화이트,이탈리아,-1,4,-1
4,24900,SET)빌라엠비앙코 + 글라스2개 윈터패키지,SET)VILLA M Bianco + GLASS WINTER PACKAGE,3.9,https://www.winenara.com/shop/product/product_...,[https://www.winenara.com/uploads/product/550/...,https://www.vivino.com/villa-m-bianco/w/1774733,디저트,이탈리아,4,-1,-1


In [128]:
winenara_attribute = ['price', 'name', 'en_name', 'rating', 'url', 'wine_type', 'country', 'sweetness', 'body', 'alcohol', 'img_url', 'vivino_link']
unified_attribute = ['url', 'site_name', 'price', 'name', 'en_name', 'img_url', 'body', 'acidity', 'tannin', "sweetness", "alcohol", "wine_type", 'country', 'grape', 'rating', 'pickup_location', 'vivino_link']
unified_dict = {'url':'url', 'site_name':'', 'price':'price', 'name':'name', 'en_name':'en_name', 'img_url':'img_url', 'body':'body', 'acidity':'', 'tannin':'', "sweetness":'sweetness', "alcohol":'alcohol', "wine_type":'wine_type', 'country':'country', 'grape':'', 'rating':'rating', 'pickup_location':'', 'vivino_link':'vivino_link'}
data_list = []
for idx in range(len(df)):
    single_data = {}
    for unified_attr, winenara_attr in unified_dict.items():
        if winenara_attr != '':
            if type(df[winenara_attr][idx]) == list:
                single_data[unified_attr] = ', '.join(df[winenara_attr][idx])
            else:
                single_data[unified_attr] = df[winenara_attr][idx]            
        elif unified_attr == 'site_name':
            single_data[unified_attr] = 'winenara'
        else:
            single_data[unified_attr] = ''
    data_list.append(single_data)

아래는 feature_img로 부터 뽑은 feature(./winenara_scrapy/assets/ocr_results)를 data_list에 추가하는 코드입니다.

In [130]:
ocr_file_list = os.listdir('./winenara_scrapy/assets/ocr_results/')

In [133]:
ocr_file_list[0][:-4]

'14핸즈 메를로.'

In [135]:
ocr_file_root_path = './winenara_scrapy/assets/ocr_results/'
ocr_file_list = os.listdir(ocr_file_root_path)

for data in data_list:
    if data['name']+'.json' in ocr_file_list:
        with open(os.path.join(ocr_file_root_path, data['name']+'.json'), encoding='utf-8') as f:
            ocr_result = json.load(f)
        print(ocr_result)

{'대표 맛과 향': '피노 블렌딩으로 붉은 과실 캐릭터가 지배적으로 느껴지며, 무수히 피어나는 수많은 꽃의 산뜻하고 화사한 향기를 느낄 수 있습니다. 와인을 즐기는 모든 순간이 즐거움으로 물들이는 플로럴 스파클링 노트를 선사합니다. 섬세하고 우아한 피니쉬 까지 깔끔하게 즐기기 좋은 스타일의 샴페인입니다.', '핸들링': '적정온도 6-7°C 아로마 와인잔 콜렉터 디캔팅 NO', '와이너리이야기': '슐룸베르거 빈, 오스트리아 Schlumberger in 1842 오스트리아 최초 샴페인 방식 양조자의 이름으로 만들 어진 최대 규모 회사며 샴페인 Ruinart에서 샴페인 양조 기술을 배워 탄생시킨 오스트리아 젝트의 원조로 유럽 내 젝트 판매 1위를 기록하고 있습니다.'}
{'대표 맛과 향': "잘 익은 사과, 감귤, 배와 같은 풍부한 과일향의 아로마가 감도는 와인입니다. 라벨에 자유의 여신상을 거대하게 그려냈 듯, 이 와인의 바디감 또한 '거대하다'고 부를만하며 누구나 편하게 즐길 수 있습니다.", '핸들링': '적정온도 10-13 °C 와인잔 화이트 디캔팅 NO 적정시기 2020-2040', '페어링': '돼지고기 생선요리'}
{'대표 맛과 향': '강렬하게 반짝이는 금빛으로 아몬드의 쌉싸름한 향, 바나나향, 스파이시한 향이 복합적으로 펼쳐지고 옅은 바닐라향, 스모키함이 뒤따릅니다. 입에선 풀바디감과 풍만하고 부드러운 과일 맛과 크리미함이 느껴집니다. 지역을 대표하는 프리울라노를 주 품종으로 우아한 샤르도네와 아로마가 풍부한 소비뇽블랑을 블렝딩한 콜리오 지역의 클래식한 화이트와인입니다.  바디     탄닌  당도  산도', '핸들링': 'X 12-13 적정온도 °C 와인잔 화이트 디캔팅 NO 적정시기 2020-2025', '페어링': '로스트 치킨 고르곤졸라 리조또'}
{}
{}
{'대표 맛과 향': '자두, 블랙베리, 블랙 커런트 향에 약간의 스파이스를 느낄 수 있으며, 입안에서는 풍부하고 부드러운 촉감과 긴 여운을 선사합니다. 미국 노예 해방의 주역 아브라

In [43]:
with open('./data/unified_wine_data.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

## 와인앤모어 크롤링

와인앤모어 크롤링을 위해서는 [Selenium](https://selenium-python.readthedocs.io/)을 사용한다.

Scrapy를 사용해서도 할 수 있다.(아마도?) 근데 나는 scrapy로 로그인과 성인인증을 뚫지 못해서 사람이 브라우저와 interaction하는 것과 비슷한 Selenium을 사용하였다.

In [4]:
import os
from time import sleep
import configparser

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

아래 코드를 실행하면 크롬이 열리고, 와인앤모어 로그인 페이지로 접속하게 된다.

In [14]:
# Setup the driver. This one uses chrome with some options and a path to the chromedriver
driver = webdriver.Chrome(ChromeDriverManager().install())

# implicitly_wait tells the driver to wait before throwing an exception
driver.implicitly_wait(30)

# go to the login page
driver.get('https://www.wineandmore.co.kr/member/login.php') # 와인앤모어 로그인페이지 접속

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 21.5MB/s]
C:\Users\chois\AppData\Local\Temp\ipykernel_17584\3443994688.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


로컬파일로부터 와인앤모어 아이디, 비밀번호 가져오기

In [10]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [11]:
wineandmore_id = config['WINE_AND_MORE']['ID']
wineandmore_password = config['WINE_AND_MORE']['PASSWORD']

아래 코드를 실행하면 아이디, 패스워드를 입력하는 부분이 채워진다.

In [15]:
# the driver.get method will navigate to a page given by the URL
username = driver.find_element_by_id("loginId")
password = driver.find_element_by_id("loginPwd")

# Target the form elements and send_keys to simulate key strokes
username.send_keys(wineandmore_id)
password.send_keys(wineandmore_password)

# Submit form
driver.find_element_by_class_name('member_login_order_btn').click()

# Allow time for the website to load
sleep(5)

C:\Users\chois\AppData\Local\Temp\ipykernel_17584\600041264.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_id("loginId")
C:\Users\chois\AppData\Local\Temp\ipykernel_17584\600041264.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_id("loginPwd")
C:\Users\chois\AppData\Local\Temp\ipykernel_17584\600041264.py:10: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('member_login_order_btn').click()


### 상품목록페이지에서 각 상품 상세페이지 링크 추출

상품리스트 페이지는 다음과 같은 url구조를 갖고있다. idx에 페이지 번호가 들어가게된다.

https://www.wineandmore.co.kr/goods/goods_list.php?page={idx}&cateCd=001

페이지는 1~28까지 있으므로 순회하며 크롤링한다.(정말 나중에는 바꿔줘야하는 안좋은 코드다..)

<img src="./assets/wineandmore_mainpage.png" width="500" height="500">

### 상품 상세페이지에서 와인 데이터 가져오기
<img src="./assets/wineanndmore_detailpage.png" width="800" height="500">

위의 이미지는 와인앤모어의 상세페이지이다. 상세페이지로부터 아래의 값들을 가져온다.
- url: 해당 페이지의 url
- name: 한글 이름
- price: 가격
- feature_img_url: 와인에 대한 상세 정보가 담겨있는 이미지 url
- img_url: 이미지 url 리스트
- pickup_info: 해당 와인의 픽업이나 구매가 가능한 와인앤모어 오프라인 매장

예를 들어 위의 이미지에 대해서는 아래와 같이 만들어진다.
```json
"url": "https://www.wineandmore.co.kr/goods/goods_view.php?goodsNo=1000002401", 
"name": "장 미셸 기불로 사비니 레 본 블랑", 
"price": "53,000", 
"feature_img_url": "https://shinsegaelnb.cdn-nhncommerce.com/data/editor/goods/230426/f2dcc9b3248c7d163cbb29c4d452cb4b_173014.jpg",
"img_url": "https://shinsegaelnb.cdn-nhncommerce.com/data/goods/23/02/08/1000002401/1000002401_detail_048.jpg", 
"pickup_info": ["와인앤모어 교대역점", "와인앤모어 다산점", "와인앤모어 서울대입구역점", "와인앤모어 스타필드 위례점", "와인앤모어 여의도점"]
```

여기서 feature_img_url이라는 것이 있는데, 이는 아래 사진과 같다. 나중에 OCR을 통해 데이터를 뽑아와야한다.

<img src="./assets/wineandmore_feature_img.png" width="500" height="500">

아래 함수는 상세페이지에서 데이터를 가져오는 코드이다.

In [ ]:
def get_detail_info(detail_url):
    driver.get(detail_url)
    # Allow time for the website to load
    sleep(10)
    data_dict = {}
    data_dict['url'] = driver.current_url

    name = driver.find_element(By.CSS_SELECTOR, ".item_detail_tit h3")
    data_dict['name'] = name.text if name else ""

    price = driver.find_element(By.CSS_SELECTOR, "dl.item_price strong strong")
    data_dict['price'] = price.text if price else ""

    data_dict['feature_img_url'] = driver.find_elements(By.CSS_SELECTOR, '.js-smart-img')[3].get_attribute('src')

    img_url = driver.find_element(By.CSS_SELECTOR, ".zoom_layer_open img.middle")
    data_dict['img_url'] = img_url.get_attribute('src')

    pickup_info_list = driver.find_elements(By.CSS_SELECTOR, ".form_element select option")
    data_dict['pickup_info'] = [pickup_info.text for pickup_info in pickup_info_list if pickup_info.text != "매장선택"]

    return data_dict

아래 코드를 실행하면 페이지를 순회하면 크롤링을 진행하고, 이를 data_list에 저장한다.

In [ ]:
# Navigate to product list page
data_list = []
for idx in range(1, 29):
    driver.get(f'https://www.wineandmore.co.kr/goods/goods_list.php?page={idx}&cateCd=001')
    sleep(10)
    product_links = driver.find_elements_by_css_selector("div.item_photo_box a")
    links = set()
    for link in product_links:
        links.add(link.get_attribute('href'))
    for detail_url in tqdm(links):
        data_list.append(get_detail_info(detail_url))

아래 코드를 통해 수집한 데이터를 [json 파일](./wine_and_more.json)로 저장한다.

In [ ]:
import json

with open('wine_data.json', 'w', encoding='utf-8') as json_file:
    for data in data_list:
        json_file.write(json.dumps(data, ensure_ascii=False) + '\n')

In [16]:
driver.close()

## CU 와인 크롤링

PocketCU라는 CU편의점 재고를 확인하는 앱이있다. 다행히 이 앱을 웹에서도 확인할 수 있다. 크게 아래의 3가지 url로 확인가능하다.
- 지도에서 아이템 재고 확인: [https://www.pocketcu.co.kr/search/stock?isRecommend=Y&item_cd={아이템id}](https://www.pocketcu.co.kr/search/stock?isRecommend=Y&item_cd=5011007015534)

<img src="./assets/cu_map.png" width=250 height=350>

- 아이템이름으로 검색: [https://www.pocketcu.co.kr/search/total/product/cubar?searchWord={아이템이름}](https://www.pocketcu.co.kr/search/total/product/cubar?searchWord=%EC%9C%84%EC%8A%A4%ED%82%A4)

<img src="./assets/cu_itemlist.png" width=250 height=350>

- 특정 매장에 어떤 아이템의 재고 확인: [https://www.pocketcu.co.kr/search/stock/storeItems/{점포id}?searchWord={아이템이름}](https://www.pocketcu.co.kr/search/stock/storeItems/04456?searchWord=%EC%A0%9C%EC%9E%84%EC%8A%A8)

<img src="./assets/cu_detailpage.png" width=250 height=350>